In [1]:
 #creating  data
import cv2
import numpy as np
import os
import string

# =========================
# CONFIG (edit these)
# =========================
mode = "trainingData"  # or "testingData"
base_dir = os.path.join("dataSet", mode)

IMG_SIZE = 224

minValue = 35                 # used in final threshold + Otsu
blur_kernel = (5, 5)          # Gaussian blur kernel (odd, odd)
blur_sigma = 2                # Gaussian sigma (strength)
adapt_block = 11              # adaptiveThreshold block size (odd)
adapt_C = 2                   # adaptiveThreshold constant

# ROI box (same idea as your original: right half of the frame)
ROI_TOP = 10

# =========================
# Helpers
# =========================
def ensure_folders():
    os.makedirs(base_dir, exist_ok=True)
    for ch in string.ascii_uppercase:
        os.makedirs(os.path.join(base_dir, ch), exist_ok=True)
    os.makedirs(os.path.join(base_dir, "SPACE"), exist_ok=True)

def next_index(folder_path: str) -> int:
    # Counts images already in folder
    files = [f for f in os.listdir(folder_path) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    return len(files)

def preprocess_roi(roi_bgr):
    """Return processed binary image (224x224) like your blur+threshold pipeline."""
    gray = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)

    # Gaussian blur (kernel + sigma)
    blur = cv2.GaussianBlur(gray, blur_kernel, blur_sigma)

    # Adaptive threshold (local thresholding)
    th = cv2.adaptiveThreshold(
        blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        adapt_block,
        adapt_C
    )

    # Final threshold with Otsu (like your script)
    _, out = cv2.threshold(th, minValue, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Resize to CNN input size
    out = cv2.resize(out, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    return out

def draw_counts(frame):
    y = 25
    cv2.putText(frame, f"Mode: {mode}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    y += 25

    # show counts for A, B, C... quickly
    for ch in "ABCDE":
        folder = os.path.join(base_dir, ch)
        cv2.putText(frame, f"{ch}: {next_index(folder)}", (10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
        y += 22

    sp = next_index(os.path.join(base_dir, "SPACE"))
    cv2.putText(frame, f"SPACE: {sp}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

# =========================
# Main
# =========================
def main():
    ensure_folders()

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Could not open webcam.")

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        frame = cv2.flip(frame, 1)  # mirror like your script :contentReference[oaicite:4]{index=4}

        h, w = frame.shape[:2]
        x1 = int(0.5 * w)
        y1 = ROI_TOP
        x2 = w - 10
        y2 = int(0.5 * w)

        # Draw ROI box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

        roi = frame[y1:y2, x1:x2]
        processed = preprocess_roi(roi)

        draw_counts(frame)

        cv2.imshow("Camera (ROI box)", frame)
        cv2.imshow("Processed (saved to disk)", processed)

        key = cv2.waitKey(10) & 0xFF
        if key == 27:  # ESC
            break

        # SPACE class (spacebar)
        if key == 32:
            folder = os.path.join(base_dir, "SPACE")
            idx = next_index(folder)
            out_path = os.path.join(folder, f"{idx}.png")
            cv2.imwrite(out_path, processed)
            print("Saved:", out_path)

        # Letters a-z
        if ord('a') <= key <= ord('z'):
            ch = chr(key).upper()
            folder = os.path.join(base_dir, ch)
            idx = next_index(folder)
            out_path = os.path.join(folder, f"{idx}.png")
            cv2.imwrite(out_path, processed)
            print("Saved:", out_path)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Saved: dataSet\trainingData\A\0.png
Saved: dataSet\trainingData\A\1.png
Saved: dataSet\trainingData\Q\1.png
